In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
sns.set(rc={'figure.figsize':(15, 10)})

In [142]:
df = pd.read_excel('gmo_analysis_data.xlsx', sheet_name=1).set_index('Date').dropna()
df.tail()

,DP,EP,US10Y
Date,,,
2022-06-30,1.64,5.23,2.98
2022-07-31,1.65,4.91,2.67
2022-08-31,1.56,5.00,3.15
2022-09-30,1.70,5.52,3.83
2022-10-31,1.75,4.97,4.10


In [143]:
rf = pd.read_excel('gmo_analysis_data.xlsx', sheet_name=3).set_index('Date')
rf.tail()

,US3M
Date,
2022-06-30,0.0014
2022-07-31,0.0020
2022-08-31,0.0025
2022-09-30,0.0028
2022-10-31,0.0035


In [144]:
GMO = pd.read_excel('gmo_analysis_data.xlsx', sheet_name=2).set_index('Date')
GMO_ex = GMO.dropna().subtract(rf['US3M'], axis=0).dropna()
GMO_ex.tail()

,SPY,GMWAX
Date,,
2022-06-30,-0.0839,-0.0656
2022-07-31,0.0901,0.0324
2022-08-31,-0.0433,-0.0283
2022-09-30,-0.0952,-0.0714
2022-10-31,0.0778,0.0352


In [145]:
def sum_stats(df, annual_fac=12): 
    table = pd.DataFrame(data=None) 
    table['Mean'] = df.mean() * annual_fac 
    table['Volatility'] = df.std() * np.sqrt(annual_fac) 
    table['SR'] = table['Mean'] / table['Volatility'] 
    return table 


In [146]:
sum_stats(GMO_ex.drop(columns=['SPY'])[:'2011'])

,Mean,Volatility,SR
GMWAX,0.0158,0.125,0.1266


In [147]:
sum_stats(GMO_ex.drop(columns=['SPY'])['2012':])

,Mean,Volatility,SR
GMWAX,0.0366,0.092,0.3982


In [148]:
sum_stats(GMO_ex.drop(columns=['SPY']))

,Mean,Volatility,SR
GMWAX,0.0245,0.1123,0.2181


In [149]:
def tail_risk(df):
    tr_df = pd.DataFrame(data = None)
    tr_df['Min return'] = df.min()
    tr_df['VaR-5th'] = df.quantile(.05)
    cum_ret = (1 + df).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max) / rolling_max
    tr_df['Max Drawdown'] = drawdown.min()
    
    return tr_df

In [150]:
tail_risk(GMO_ex.drop(columns=['SPY'])[:'2011'])

,Min return,VaR-5th,Max Drawdown
GMWAX,-0.1492,-0.0598,-0.4729


In [151]:
tail_risk(GMO_ex.drop(columns=['GMWAX'])[:'2011'])

,Min return,VaR-5th,Max Drawdown
SPY,-0.1656,-0.0802,-0.56


In [152]:
tail_risk(GMO_ex.drop(columns=['SPY'])['2012':])

,Min return,VaR-5th,Max Drawdown
GMWAX,-0.1187,-0.0397,-0.226


In [153]:
tail_risk(GMO_ex.drop(columns=['GMWAX'])['2012':])

,Min return,VaR-5th,Max Drawdown
SPY,-0.1247,-0.0687,-0.2481


In [154]:
tail_risk(GMO_ex.drop(columns=['SPY']))

,Min return,VaR-5th,Max Drawdown
GMWAX,-0.1492,-0.0483,-0.4729


In [155]:
tail_risk(GMO_ex.drop(columns=['GMWAX']))

,Min return,VaR-5th,Max Drawdown
SPY,-0.1656,-0.08,-0.56


(a) GMO has lower tail risk than SPY. Across all periods examined it has lower VaR, max drawdown, and minimum return.

(b) Yes, GMO's VaR improves notably in the second subsample, and drawdown is also much lower in the second subsample.

In [156]:
reg1 = sm.OLS(GMO_ex.drop(columns=['SPY'])[:'2011'], sm.add_constant(GMO_ex.drop(columns=['GMWAX'])[:'2011'])).fit() 
reg2 = sm.OLS(GMO_ex.drop(columns=['SPY'])['2012':], sm.add_constant(GMO_ex.drop(columns=['GMWAX'])['2012':])).fit() 
reg3 = sm.OLS(GMO_ex.drop(columns=['SPY']), sm.add_constant(GMO_ex.drop(columns=['GMWAX']))).fit() 
reg_report = pd.DataFrame(data=None, index=['Alpha', 'Beta', 'R-square']) 
reg_report['Inception - 2011'] = [reg1.params[0] * 12, reg1.params[1], reg1.rsquared] 
reg_report['2012 - Present'] = [reg2.params[0] * 12, reg2.params[1], reg2.rsquared] 
reg_report['Full Sample'] = [reg3.params[0] * 12, reg3.params[1], reg3.rsquared] 
reg_report

,Inception - 2011,2012 - Present,Full Sample
Alpha,-0.0058,-0.0345,-0.0170
Beta,0.5396,0.5622,0.5456
R-square,0.5071,0.7645,0.5777


(b) While GMO has moderate exposure to the market and it's market beta is not very low, we can consider it a low-beta strategy. The beta is consistent across the subsamples so GMO's exposure to the market has not changed.

(c) GMO does not provide alpha in either subsample as alpha is negative.

In [157]:
df = df.shift()
df['SPY'] = GMO['SPY']
df.head()

,DP,EP,US10Y,SPY
Date,,,,
1993-02-28,NaN,NaN,NaN,0.0107
1993-03-31,2.82,4.44,6.03,0.0224
1993-04-30,2.77,4.41,6.03,-0.0256
1993-05-31,2.82,4.44,6.05,0.0270
1993-06-30,2.81,4.38,6.16,0.0037


In [158]:
def reg_params(df, y_col, X_col, intercept = True, annual_fac=12):
    y = df[y_col]
    if intercept == True:
        X = sm.add_constant(df[X_col])
    else:
        X = df[X_col]
    
    model = sm.OLS(y, X, missing = 'drop').fit()
    reg_df = model.params.to_frame('Regression Parameters')
    reg_df.loc[r"$R^{2}$"] = model.rsquared
    
    if intercept == True:
        reg_df.loc['const'] *= annual_fac
    
    return reg_df

In [159]:
DP = reg_params(df, 'SPY', 'DP')
DP

,Regression Parameters
const,-0.1129
DP,0.0094
$R^{2}$,0.0094


In [160]:
EP = reg_params(df, 'SPY', 'EP')
EP

,Regression Parameters
const,-0.0712
EP,0.0032
$R^{2}$,0.0086


In [161]:
EP_DP_10Y = reg_params(df, 'SPY', ['EP','DP','US10Y'])
EP_DP_10Y 

,Regression Parameters
const,-0.1792
EP,0.0027
DP,0.0080
US10Y,-0.0010
$R^{2}$,0.0163


In [162]:
w_DP = 100 * (DP.loc['const'][0]/12 + DP.loc['DP'][0] * df['DP'])
r_DP = (w_DP * df['SPY']).dropna()
w_EP = 100 * (EP.loc['const'][0]/12 + EP.loc['EP'][0] * df['EP'])
r_EP = (w_EP * df['SPY']).dropna()
w_3fac = 100 * (EP_DP_10Y.loc['const'][0]/12 + EP_DP_10Y.loc['EP'][0] * df['EP']\
                                             + EP_DP_10Y.loc['DP'][0] * df['DP']\
                                             + EP_DP_10Y.loc['US10Y'][0] * df['US10Y'])
r_3fac = (w_3fac * df['SPY']).dropna() 
ret_table = pd.DataFrame(data=None) 
ret_table['DP'] = r_DP 
ret_table['EP'] = r_EP 
ret_table['Three Factors'] = r_3fac
ret_table

,DP,EP,Three Factors
Date,,,
1993-03-31,0.0385,0.0185,0.0303
1993-04-30,-0.0428,-0.0209,-0.0333
1993-05-31,0.0464,0.0223,0.0364
1993-06-30,0.0063,0.0030,0.0048
1993-07-31,-0.0082,-0.0038,-0.0064
...,...,...,...
2022-06-30,-0.0445,-0.0773,-0.0621
2022-07-31,0.0558,0.0993,0.0841
2022-08-31,-0.0251,-0.0398,-0.0354


In [163]:
def summary_stats_bm(series, bm, annual_fac=12):
    ss_df = pd.DataFrame(data = None, index = ['Summary Stats'])
    ss_df['Mean'] = series.mean() * annual_fac
    ss_df['Vol'] = series.std() * np.sqrt(annual_fac)
    ss_df['Sharpe (Mean/Vol)'] = ss_df['Mean'] / ss_df['Vol']
    
    y = series
    X = sm.add_constant(bm.loc[series.index])
    reg = sm.OLS(y,X).fit()
    resid = reg.resid
    reg = sm.OLS(y,X).fit().params
    ss_df['alpha'] = reg[0] * annual_fac
    ss_df['beta'] = reg[1] 
    ss_df['Information Ratio'] = (reg[0] / resid.std()) * np.sqrt(annual_fac)
    
    cum_ret = (1 + series).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max) / rolling_max
    ss_df['Max Drawdown'] = drawdown.min()
    
    return round(ss_df, 4)

In [164]:
summary_stats_bm(ret_table['DP'], df['SPY'])

,Mean,Vol,Sharpe (Mean/Vol),alpha,beta,Information Ratio,Max Drawdown
Summary Stats,0.1095,0.149,0.7348,0.0207,0.8611,0.2759,-0.653


In [165]:
summary_stats_bm(ret_table['EP'], df['SPY'])

,Mean,Vol,Sharpe (Mean/Vol),alpha,beta,Information Ratio,Max Drawdown
Summary Stats,0.1078,0.1286,0.8383,0.0322,0.7327,0.4789,-0.3823


In [166]:
summary_stats_bm(ret_table['Three Factors'], df['SPY'])

,Mean,Vol,Sharpe (Mean/Vol),alpha,beta,Information Ratio,Max Drawdown
Summary Stats,0.125,0.1456,0.8588,0.0451,0.775,0.5118,-0.5221


In [167]:
VaR = pd.DataFrame([r_DP.quantile(.05), r_EP.quantile(.05), r_3fac.quantile(.05), 
                    df['SPY'].quantile(.05), 
                    GMO['GMWAX'].quantile(.05)],
                   index = ['DP Strat','EP Strat','3-factor Strat','SPY','GMO'], 
                   columns = ['5% VaR'])
VaR

,5% VaR
DP Strat,-0.0523
EP Strat,-0.0541
3-factor Strat,-0.0642
SPY,-0.0739
GMO,-0.0473


In [168]:
sum_stats(ret_table.loc['2000':'2011', ['DP']])


,Mean,Volatility,SR
DP,0.0393,0.1842,0.2135


In [169]:
sum_stats(ret_table.loc['2000':'2011', ['EP']])

,Mean,Volatility,SR
EP,0.0373,0.1339,0.2784


In [170]:
sum_stats(ret_table.loc['2000':'2011', ['Three Factors']])

,Mean,Volatility,SR
Three Factors,0.0608,0.1574,0.3863


In [171]:
sum_stats(rf.loc['2000':'2011'])

,Mean,Volatility,SR
US3M,0.0231,0.0058,3.9866


(b) All the dynamic strategies outperform the risk-free rate during this period.

In [172]:
ret_table['rf'] = rf['US3M']
df_riskprem = pd.DataFrame(data=None, index=['% of periods underperforming Rf'])
for col in ret_table.columns[:3]:
    df_riskprem[col] = len(ret_table[ret_table[col] < ret_table['rf']])/len(ret_table) * 100
    
df_riskprem

,DP,EP,Three Factors
% of periods underperforming Rf,37.3596,37.3596,37.0787


(d) No, judging by the tail risk metrics and volatility of the dynamic strategies compared to SPY it does not seem like these strategies take on extra risk on the whole. However, we must keep in mind that the strategies are dependent on running regressions with very little prediction power, so badly estimated parameters could lead to terrible performance. This is not evident in terms of very high volatility and tail risk in our backtesting period though.

In [173]:
def OOS_r2(df, factors, start):
    y = df['SPY']
    X = sm.add_constant(df[factors])

    forecast_err, null_err = [], []

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            forecast_err.append(reg_predict - actual)
            null_err.append(null_forecast - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    
    return 1 - RSS/TSS

In [174]:
EP_OOS_r2 = OOS_r2(df, ['EP'], 60)

print('EP OOS R-squared: ' + str(round(EP_OOS_r2, 4)))

EP OOS R-squared: -0.007


No, the R^2 value is negative.

In [175]:
def OOS_strat(df, factors, start, weight):
    returns = []
    y = df['SPY']
    X = sm.add_constant(df[factors])

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            pred = reg.predict(X.iloc[[i]])
            w = pred * weight
            returns.append((df.iloc[i]['SPY'] * w)[0])

    df_strat = pd.DataFrame(data = returns, index = df.iloc[-(len(returns)):].index, columns = ['Strat Returns'])
    return df_strat

In [176]:
OOS_EP = OOS_strat(df, ['EP'], 60, 100)
OOS_EP.head()

,Strat Returns
Date,
1998-02-28,0.0512
1998-03-31,0.0456
1998-04-30,0.0143
1998-05-31,-0.0225
1998-06-30,0.0351


In [177]:
summary_stats_bm(OOS_EP['Strat Returns'], GMO[['SPY']])

,Mean,Vol,Sharpe (Mean/Vol),alpha,beta,Information Ratio,Max Drawdown
Summary Stats,0.0819,0.1654,0.4953,0.0353,0.5435,0.249,-0.5837


In [178]:
VaR_OOS = pd.DataFrame([OOS_EP['Strat Returns'].quantile(.05),  
                    df['SPY'].quantile(.05), 
                    GMO['GMWAX'].quantile(.05)],
                   index = ['EP Strat','SPY','GMO'], 
                   columns = ['5% VaR'])

VaR_OOS

,5% VaR
EP Strat,-0.0710
SPY,-0.0739
GMO,-0.0473


In [179]:
summary_stats_bm(OOS_EP.loc['2000':'2011']['Strat Returns'], GMO[['SPY']])

,Mean,Vol,Sharpe (Mean/Vol),alpha,beta,Information Ratio,Max Drawdown
Summary Stats,0.0388,0.1959,0.1979,0.0333,0.2994,0.1757,-0.5837


In [180]:
sum_stats(rf.loc['2000':'2011'])

,Mean,Volatility,SR
US3M,0.0231,0.0058,3.9866


In [181]:
r_df_OOS = OOS_EP.rename(columns={"Strat Returns": "EP Strat"})
r_df_OOS['rf'] = rf['US3M']

df_riskprem2 = pd.DataFrame(data=None, index=[r'% of periods underperforming $r^{f}$'])
for col in r_df_OOS.columns[:3]:
    df_riskprem2[col] = len(r_df_OOS[r_df_OOS[col] < r_df_OOS['rf']])/len(r_df_OOS) * 100
    
df_riskprem2

,EP Strat,rf
% of periods underperforming $r^{f}$,38.3838,0.0


(d) The dynamic strategy tends to have worse risk metrics than SPY so it seems this strategy does take on extra risk.